In [558]:
import notebook_common
import jaconv
import os
from importlib import reload

reload(notebook_common)
from notebook_common import (
    # ------------------function------------------
    # read_excel,
    to_csv_sig,
    # ------------------2400 word------------------
    # r_word_2400_xlsx,
    # w_word_2400_csv,
    # w_word_2400_with_guide_csv,
    w_word_2400_without_guide_csv,
    # ------------------书面语相关------------------
    # 解析书面语正则
    # pattern_table_written,
    # pattern_td_written,
    # r_written_freq_nlt_with_enter_txt,
    # w_written_freq_nlt_with_enter_csv,
    w_written_freq_nlt_sumed_csv,
    # ------------------口语相关------------------
    # 解析口头语正则
    # pattern_table_spoken,
    # pattern_td_spoken,
    # r_spoken_freq_nlb_with_enter_txt,
    # w_spoken_freq_nlb_with_enter_csv,
    # w_spoken_freq_nlb_with_enter_filtered_csv,
    # w_spoken_freq_nlb_with_enter_filtered_hrt_csv,
    w_spoken_freq_nlb_sumed_csv,
    # ------------------合并书面语和口语词频------------------
    w_merged_freq_sumed_csv,
    w_raw_merged_freq_sumed_csv,
    w_word_raw_merged_freq_sumed_csv,
    # ------------------公共python库------------------
    pd,
    np,
    p,
    l,
)

p("Import common module")

Import common module


In [559]:
df_spoken = pd.read_csv(
    w_spoken_freq_nlb_sumed_csv,
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
    index_col=["headword", "hira"],
)

df_written = pd.read_csv(
    w_written_freq_nlt_sumed_csv,
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
    index_col=["headword", "hira"],
)


df_w_word_2400_without_guide_csv = pd.read_csv(
    w_word_2400_without_guide_csv,
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
    index_col=0,
)
l("df_spoken")
df_spoken

----------------------------------------df_spoken----------------------------------------


,,spoken_freq
headword,hira,
の,の,57221425
に,に,35286930
を,を,33707106
は,は,31279365
が,が,30705135
...,...,...
ｋワット,きろわっと,1
右顧,うこ,1
右脳する,うのうする,1


In [560]:
l("df_written")
df_written

----------------------------------------df_written----------------------------------------


,,roman,written_freq
headword,hira,,
いる,いる,iru,1067608
こと,こと,koto,701470
の,の,no,686025
する,する,suru,607405
なる,なる,naru,528539
...,...,...,...
焼きいもする,やきいもする,yakiimosuru,1
へろへろする,へろへろする,heroherosuru,1
へなへなする,へなへなする,henahenasuru,1


In [561]:
l("df_w_word_2400_without_guide_csv")
df_w_word_2400_without_guide_csv

----------------------------------------df_w_word_2400_without_guide_csv----------------------------------------


,word,hira,pos,prop1,sub_prop1,...,sub_prop3,remark,meaning,cnt-word,cnt-word-hira
number,,,,,,,,,,,
1,ああ,ああ,NaN,感叹词,NaN,...,NaN,NaN,啊、嗯、啊呀、唉,1,1
2,愛,あい,五十音,名词,NaN,...,NaN,他,爱情、爱好,1,1
3,あいさつ,あいさつ,NaN,名词,NaN,...,NaN,自,寒暄、致辞,1,1
4,合図,あいず,NaN,名词,NaN,...,NaN,NaN,信号,1,1
5,アイスクリーム,あいすくりーむ,NaN,名词,NaN,...,NaN,NaN,冰激凌、雪糕,1,1
...,...,...,...,...,...,...,...,...,...,...,...
2460,象,ぞう,Ｌ13-2,名词,NaN,...,NaN,NaN,大象,1,1
2461,東京,とうきょう,Ｌ13-2,名词,NaN,...,NaN,NaN,东京,1,1
2462,居間,いま,Ｌ14,名词,NaN,...,NaN,NaN,起居室，起坐间,1,1


In [562]:
# 输出口语和书面语交集，索引是联合索引 拼写-发音
ids_df_spoken = set(df_spoken.index)
p(f"spoken freq index len:{len(ids_df_spoken)}")
ids_df_written = set(df_written.index)
p(f"written freq index len:{len(ids_df_written)}")
common_ids = ids_df_spoken & ids_df_written
p(f"common index len:{len(common_ids)}")

spoken freq index len:96948
written freq index len:84599
common index len:77302


In [563]:
# 原始书面语和口语词频合并
df_merged_raw = pd.concat([df_spoken, df_written], axis=1, join="outer")

# 空值补0
df_merged_raw["spoken_freq"] = df_merged_raw["spoken_freq"].fillna(0)
df_merged_raw["written_freq"] = df_merged_raw["written_freq"].fillna(0)

df_merged_raw["cnt_headword_hira"] = df_merged_raw.groupby(
    ["headword", "hira"]
).transform("size")
df_merged_raw["cnt_headword"] = df_merged_raw.groupby(["headword"]).transform("size")


# to_csv_sig(df_merged_raw, w_raw_merged_freq_sumed_csv, True)
p("df_merged_raw--ok")
# df_merged_raw

df_merged_raw--ok


In [564]:
# 作标准化处理 ，统计计算每4294,0640的词频, 以书面语词频作为基准
sum_spoken_freq = df_merged_raw["spoken_freq"].sum()
sum_written_freq = df_merged_raw["written_freq"].sum()
p(f"sum_spoken_freq:{sum_spoken_freq}")
p(f"sum_written_freq:{sum_written_freq}")
base_spoken_rate = sum_spoken_freq / sum_written_freq
# base_written_rate = sum_written_freq/10000000
p(f"base_spoken_rate:{base_spoken_rate}")
# p(base_written_rate)

sum_spoken_freq:949782694.0
sum_written_freq:42940640.0
base_spoken_rate:22.118503450344477


In [565]:
# 关联mp3发音文件
if os.path.exists("d:/tmp/df_ff_5.csv"):
    df_ff_5 = pd.read_csv(
        "d:/tmp/df_ff_5.csv", encoding="utf-8-sig", on_bad_lines="skip", engine="python"
    )
    l("ok")
else:
    df_ff_4 = pd.read_csv(
        "D:/04.japandict/Forvo/ff-4.csv",
        encoding="utf-8-sig",
        on_bad_lines="skip",
        engine="python",
    )
    df_ff_4.dropna(subset=["headword_voice", "mp3_path"], inplace=True)

    df_ff_4["headword_voice"] = df_ff_4["headword_voice"].apply(jaconv.kata2hira)

    df_ff_5 = (
        df_ff_4.groupby("headword_voice")
        .agg({"mp3_path": lambda x: "|".join(x[x != ""])})  # 字符串拼接  # 求平均值
        .reset_index()
    )
    l("df_ff_5")
    to_csv_sig(df_ff_5, "d:/tmp/df_ff_5.csv")

----------------------------------------ok----------------------------------------


In [566]:
df_merged_raw

,,spoken_freq,roman,written_freq,cnt_headword_hira,cnt_headword
headword,hira,,,,,
の,の,57221425.0,no,686025.0,1,1
に,に,35286930.0,NaN,0.0,1,1
を,を,33707106.0,NaN,0.0,1,1
は,は,31279365.0,NaN,0.0,1,1
が,が,30705135.0,NaN,0.0,1,1
...,...,...,...,...,...,...
鼽衄,きゅうじく,0.0,kyuujiku,1.0,1,1
8ミリビデオカメラ,はちみりびでおかめら,0.0,hachimiribideokamera,1.0,1,1
焦がれ死ぬ,こがれしぬ,0.0,kogareshinu,1.0,1,1


In [567]:
# 对词频做标准化处理

# df_merged_raw["raw_spoken_freq"] = df_merged_raw["spoken_freq"]
# df_merged_raw["raw_written_freq"] = df_merged_raw["written_freq"]

# df_merged_raw["spoken_freq"] = df_merged_raw["spoken_freq"] / base_spoken_rate


# df_spoken_written_freq_headword
spoken_weight, written_weight = 1, 1

df_merged_raw["fused_freq"] = round(
    df_merged_raw["spoken_freq"] * spoken_weight / base_spoken_rate
    + df_merged_raw["written_freq"] * written_weight
)


df_merged_raw["freq_for_order"] = round(
    df_merged_raw["spoken_freq"] * spoken_weight
    + df_merged_raw["written_freq"] * base_spoken_rate * written_weight
)


# 计算发音的词频占比
df_merged_raw["freq_sum_headword"] = df_merged_raw.groupby("headword")[
    "fused_freq"
].transform("sum")
df_merged_raw["hira_freq_per"] = round(
    df_merged_raw["fused_freq"] / df_merged_raw["freq_sum_headword"], 2
)


# #过滤融合词频>5的
# df_merged_raw=df_merged_raw[df_merged_raw['fused_freq']>1]
# p(len(df_merged_raw))

# 生成平假名发音id，2位数字，从10开始
df_merged_raw = df_merged_raw.sort_values(
    ["headword", "fused_freq"], ascending=[True, False]
)
df_merged_raw["hira_order"] = df_merged_raw.groupby("headword").cumcount() + 10
df_merged_raw = df_merged_raw[df_merged_raw["hira_order"] < 100]
df_merged_raw["hira_order"] = df_merged_raw["hira_order"].astype(str).str.zfill(2)
df_merged_raw.sort_values(by="freq_for_order", ascending=False, inplace=True)


# to_csv_sig(df_merged_raw,"d:/tmp/hira_order.csv",True)


l("ok")

----------------------------------------ok----------------------------------------


In [568]:
df_merged_raw

,,spoken_freq,roman,written_freq,cnt_headword_hira,cnt_headword,fused_freq,freq_for_order,freq_sum_headword,hira_freq_per,hira_order
headword,hira,,,,,,,,,,
の,の,57221425.0,no,686025.0,1,1,3273064.0,72395271.0,3273064.0,1.0,10
いる,いる,12277963.0,iru,1067608.0,1,1,1622707.0,35891854.0,1622707.0,1.0,10
に,に,35286930.0,NaN,0.0,1,1,1595358.0,35286930.0,1595358.0,1.0,10
を,を,33707106.0,NaN,0.0,1,1,1523932.0,33707106.0,1523932.0,1.0,10
は,は,31279365.0,NaN,0.0,1,1,1414172.0,31279365.0,1414172.0,1.0,10
...,...,...,...,...,...,...,...,...,...,...,...
錆竹,さびだけ,1.0,NaN,0.0,1,1,0.0,1.0,0.0,NaN,10
錯落,さくらく,1.0,NaN,0.0,1,1,0.0,1.0,0.0,NaN,10
鍔迫り合いする,つばぜりあいする,1.0,NaN,0.0,1,1,0.0,1.0,0.0,NaN,10


In [569]:
df_merged_raw_word = df_merged_raw.reset_index(level=1, drop=True)
df_merged_raw_word

,spoken_freq,roman,written_freq,cnt_headword_hira,cnt_headword,fused_freq,freq_for_order,freq_sum_headword,hira_freq_per,hira_order
headword,,,,,,,,,,
の,57221425.0,no,686025.0,1,1,3273064.0,72395271.0,3273064.0,1.0,10
いる,12277963.0,iru,1067608.0,1,1,1622707.0,35891854.0,1622707.0,1.0,10
に,35286930.0,NaN,0.0,1,1,1595358.0,35286930.0,1595358.0,1.0,10
を,33707106.0,NaN,0.0,1,1,1523932.0,33707106.0,1523932.0,1.0,10
は,31279365.0,NaN,0.0,1,1,1414172.0,31279365.0,1414172.0,1.0,10
...,...,...,...,...,...,...,...,...,...,...
錆竹,1.0,NaN,0.0,1,1,0.0,1.0,0.0,NaN,10
錯落,1.0,NaN,0.0,1,1,0.0,1.0,0.0,NaN,10
鍔迫り合いする,1.0,NaN,0.0,1,1,0.0,1.0,0.0,NaN,10


In [570]:
# 词汇以headword分组，统计
df_merged_raw_word_agg = (
    df_merged_raw_word.groupby(df_merged_raw_word.index)
    .agg(
        {
            "fused_freq": lambda x: sum(x),
            "spoken_freq": lambda x: sum(x),
            "written_freq": lambda x: sum(x),
            "freq_for_order": lambda x: sum(x),
        }
    )  # 字符串拼接  # 求平均值
    .reset_index()
)
l("ok")

----------------------------------------ok----------------------------------------


In [571]:
# 输出以word为基础的词频表
df_merged_raw_word_agg.sort_values(by="freq_for_order", ascending=False, inplace=True)
start_index = 100000
df_merged_raw_word_agg.index = range(
    start_index, start_index + len(df_merged_raw_word_agg)
)
to_csv_sig(df_merged_raw_word_agg, w_word_raw_merged_freq_sumed_csv, True)
l("df_merged_raw_word_agg--ok")
df_merged_raw_word_agg

----------------------------------------df_merged_raw_word_agg--ok----------------------------------------


,headword,fused_freq,spoken_freq,written_freq,freq_for_order
100000,の,3273064.0,57221425.0,686025.0,72395271.0
100001,いる,1622707.0,12277963.0,1067608.0,35891854.0
100002,に,1595358.0,35286930.0,0.0,35286930.0
100003,を,1523932.0,33707106.0,0.0,33707106.0
100004,は,1414172.0,31279365.0,0.0,31279365.0
...,...,...,...,...,...
202300,入念する,0.0,1.0,0.0,1.0
202301,さぶする,0.0,1.0,0.0,1.0
202302,8する,0.0,1.0,0.0,1.0
202303,進士する,0.0,1.0,0.0,1.0


In [572]:
df_merged_raw

,,spoken_freq,roman,written_freq,cnt_headword_hira,cnt_headword,fused_freq,freq_for_order,freq_sum_headword,hira_freq_per,hira_order
headword,hira,,,,,,,,,,
の,の,57221425.0,no,686025.0,1,1,3273064.0,72395271.0,3273064.0,1.0,10
いる,いる,12277963.0,iru,1067608.0,1,1,1622707.0,35891854.0,1622707.0,1.0,10
に,に,35286930.0,NaN,0.0,1,1,1595358.0,35286930.0,1595358.0,1.0,10
を,を,33707106.0,NaN,0.0,1,1,1523932.0,33707106.0,1523932.0,1.0,10
は,は,31279365.0,NaN,0.0,1,1,1414172.0,31279365.0,1414172.0,1.0,10
...,...,...,...,...,...,...,...,...,...,...,...
錆竹,さびだけ,1.0,NaN,0.0,1,1,0.0,1.0,0.0,NaN,10
錯落,さくらく,1.0,NaN,0.0,1,1,0.0,1.0,0.0,NaN,10
鍔迫り合いする,つばぜりあいする,1.0,NaN,0.0,1,1,0.0,1.0,0.0,NaN,10


In [573]:
# df_merged_raw 关联 df_merged_raw_word_agg的ID主键
df_merged_raw_word_agg.reset_index(inplace=True)
df_merged_raw_word_agg = df_merged_raw_word_agg.rename(columns={"index": "idx"})
df_merged_raw_word_agg

,idx,headword,fused_freq,spoken_freq,written_freq,freq_for_order
0,100000,の,3273064.0,57221425.0,686025.0,72395271.0
1,100001,いる,1622707.0,12277963.0,1067608.0,35891854.0
2,100002,に,1595358.0,35286930.0,0.0,35286930.0
3,100003,を,1523932.0,33707106.0,0.0,33707106.0
4,100004,は,1414172.0,31279365.0,0.0,31279365.0
...,...,...,...,...,...,...
102300,202300,入念する,0.0,1.0,0.0,1.0
102301,202301,さぶする,0.0,1.0,0.0,1.0
102302,202302,8する,0.0,1.0,0.0,1.0
102303,202303,進士する,0.0,1.0,0.0,1.0


In [574]:
# df_merged_raw 关联 df_merged_raw_word_agg的ID主键
df_merged_raw_word_agg_simple = df_merged_raw_word_agg[
    ["headword", "idx", "fused_freq"]
].copy()

df_merged_raw_word_agg_simple.rename(
    columns={"fused_freq": "word_fused_freq"}, inplace=True
)

# df_merged_raw_word_agg_simple

df_merged_raw2 = df_merged_raw.reset_index()
# df_merged_raw2

df_hira = pd.merge(
    df_merged_raw2, df_merged_raw_word_agg_simple, on="headword", how="left"
)

df_hira["idx"] = df_hira["idx"].astype(str)

df_hira["hira_idx"] = df_hira["idx"] + df_hira["hira_order"]
df_hira.sort_values(
    by=["word_fused_freq", "freq_for_order"], ascending=[False, False], inplace=True
)
to_csv_sig(df_hira, "d:/tmp/result.csv", True)
l("df_hira")

----------------------------------------df_hira----------------------------------------


In [576]:
# 关联发音文件 (headword关联)
# df_merged_raw.reset_index(inplace=True)

df_hira["word_voice"] = df_hira["headword"].map(
    df_ff_5.set_index("headword_voice")["mp3_path"]
)
df_hira["hira_voice"] = df_hira["hira"].map(
    df_ff_5.set_index("headword_voice")["mp3_path"]
)
# df_merged_raw_new = pd.merge(
#     df_hira,
#     df_ff_5,
#     left_on="headword",
#     right_on="headword_voice",
#     how="left",
# )
order = [
    "hira_idx",
    "idx",
    "headword",
    "hira",
    "hira_freq_per",
    "word_fused_freq",
    "fused_freq",
    "freq_for_order",
    "word_voice",
    "hira_voice",
    "spoken_freq",
    "written_freq",
    "roman",
    "cnt_headword",
    "cnt_headword_hira",
]

# df_merged_raw_new = df_merged_raw_new[order]
df_hira = df_hira[order]
df_hira_new = df_hira.sort_values(
    by=["word_fused_freq", "freq_for_order"], ascending=[False, False]
)
to_csv_sig(df_hira_new, w_raw_merged_freq_sumed_csv, True)
p("df_hira--ok")
df_hira_new

df_hira--ok


,hira_idx,idx,headword,hira,hira_freq_per,...,spoken_freq,written_freq,roman,cnt_headword,cnt_headword_hira
0,10000010,100000,の,の,1.0,...,57221425.0,686025.0,no,1,1
1,10000110,100001,いる,いる,1.0,...,12277963.0,1067608.0,iru,1,1
2,10000210,100002,に,に,1.0,...,35286930.0,0.0,NaN,1,1
3,10000310,100003,を,を,1.0,...,33707106.0,0.0,NaN,1,1
4,10000410,100004,は,は,1.0,...,31279365.0,0.0,NaN,1,1
...,...,...,...,...,...,...,...,...,...,...,...
104091,19708910,197089,錆竹,さびだけ,NaN,...,1.0,0.0,NaN,1,1
104092,19702110,197021,錯落,さくらく,NaN,...,1.0,0.0,NaN,1,1
104093,19690310,196903,鍔迫り合いする,つばぜりあいする,NaN,...,1.0,0.0,NaN,1,1
104094,19686010,196860,鍾する,しょうする,NaN,...,1.0,0.0,NaN,1,1


In [ ]:
"""
2400词汇跟，合并后的书面语和口语的词频表进行左连接，用new_chinese进行连接

"""


df_spoken_written_freq_headword = (
    df_merged_raw.groupby("headword")[["spoken_freq", "written_freq"]]
    .sum()
    .reset_index()
)

# 对词频做标准化处理
df_spoken_written_freq_headword["spoken_freq"] = (
    df_spoken_written_freq_headword["spoken_freq"] / base_spoken_rate
)


# df_spoken_written_freq_headword
spoken_weight = 0.5
written_weight = 0.5
df_spoken_written_freq_headword["fused_freq"] = round(
    df_spoken_written_freq_headword["spoken_freq"] * spoken_weight
    + df_spoken_written_freq_headword["written_freq"] * written_weight
)


# 先用word进行连接
# t1 = pd.merge(df_w_word_2400_without_guide_csv,df_spoken_written_freq_headword,left_on='word',right_on='headword',how='left')
# t1_renamed = t1.rename(columns={'spoken_freq': 'word_spoken_freq', 'written_freq': 'word_written_freq','headword':'word_headword'})
# 再用chinese进行连接
# t2 = pd.merge(t1_renamed,df_spoken_written_freq_headword,left_on='chinese',right_on='headword',how='left')
# t2_renamed = t2.rename(columns={'spoken_freq': 'chinese_spoken_freq', 'written_freq': 'chinese_written_freq','headword':'chinese_headword'})

t1 = pd.merge(
    df_w_word_2400_without_guide_csv,
    df_spoken_written_freq_headword,
    left_on="new_chinese",
    right_on="headword",
    how="left",
)


to_csv_sig(t1, "d:/tmp/a.csv")
p("ok")

In [ ]:
df_merged = pd.concat([df_spoken, df_written], axis=1, join="outer")


def type_dection(spoken, written):
    if spoken > 0 and written > 0:
        return "both"
    elif np.isnan(spoken):
        return "written"
    elif np.isnan(written):
        return "spoken"
    else:
        return "empty"


df_merged["type"] = df_merged.apply(
    lambda x: type_dection(x["spoken_freq"], x["written_freq"]), axis=1
)

# 查看headword重复的次数
df_merged["headword_repetition"] = df_merged.groupby(["headword"]).transform("size")

# 查看reading重复的次数
df_merged["reading_repetition"] = df_merged.groupby(["reading"]).transform("size")

# 只对headword列进行统计
df_merged["gr_by_word_sum_s_freq"] = df_merged.groupby(["headword"])[
    "spoken_freq"
].transform("sum")
df_merged["gr_by_word_sum_w_freq"] = df_merged.groupby(["headword"])[
    "written_freq"
].transform("sum")

# df_merged.index
to_csv_sig(df_merged, w_merged_freq_sumed_csv, True)
p("w_merged_freq_sumed_csv--ok")

In [ ]:
# word_in_headword  #找出2400价目词汇里有多少个在词频里
word2400_set = set(df_w_word_2400_without_guide_csv["new_chinese"])
p(len(word2400_set))

headword_freq_merge_set = set(df_merged.index.get_level_values(0))
p(len(headword_freq_merge_set))

reading_freq_merge_set = set(df_merged.index.get_level_values(1))
p(len(reading_freq_merge_set))

total_word = 0
effect = 0
word_in_headword = 0
word_in_reading = 0
for word in word2400_set:
    total_word += 1
    if word in headword_freq_merge_set:
        effect += 1
        word_in_headword += 1
        continue
    if word in reading_freq_merge_set:
        p(f"reading word:{word}")
        effect += 1
        word_in_reading += 1
        continue

p(f"effect:{effect}")
p(f"total_word:{total_word}")
p(f"word in headword:{word_in_headword}")
p(f"word in reading:{word_in_reading}")

p(effect / total_word)